<a href="https://colab.research.google.com/github/ohashi-gnct/exp/blob/2022/3%E5%B9%B4%E7%94%9F%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9_C_%E6%84%9F%E6%9F%93%E8%80%85%E6%95%B0%E3%81%A8%E4%BA%BA%E6%B5%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import io
import json
import datetime as dt

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

Colabに最初から入っていないライブラリをUNIXコマンドの`pip`で導入する。

`japanize-matplotlib`は日本語でmatplotlibが使えるようにするライブラリである。

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# matplotlibの日本語化
import japanize_matplotlib
%matplotlib inline
sns.set(font="IPAexGothic")

Googleドライブを接続し、CSVファイルを自分のドライブに保存する。

認証を求める画面が出る場合がある。

自身でJupyter notebook環境を作っている場合、ディレクトリは適当に読み替えること。

In [ ]:
# Googleドライブの接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_path = '/content/drive/MyDrive'

Mounted at /content/drive


プロットの見栄えがするように初期設定する。

In [ ]:
# matplotlibの初期設定
plt_config = {
    "figure.dpi": 150,
    "font.size": 14,
    "axes.xmargin": 0,
    "axes.ymargin": 0,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "mathtext.cal": "serif",
    "mathtext.rm": "serif",
    "mathtext.it": "serif:italic",
    "mathtext.bf": "serif.bold",
    "mathtext.fontset": "cm",
    "legend.framealpha": 1.0,
    "legend.edgecolor": "black",
    "legend.fancybox": False
}
plt.rcParams.update(plt_config)

# コロナ感染者

NHKの新型コロナウイルス特設サイトからCSVをダウンロードする。

1日ごとの全国の感染者が記録されている。

https://www3.nhk.or.jp/news/special/coronavirus/data/

UNIXコマンド`curl`でGoogleドライブにファイルをダウンロードする。

In [ ]:
!curl -o /content/drive/MyDrive/covid19_cases.csv https://www3.nhk.or.jp/n-data/opendata/coronavirus/nhk_news_covid19_domestic_daily_data.csv

In [ ]:
df_covid = pd.read_csv(os.path.join(drive_path, "covid19_cases.csv"), 
                  names=("date", "cases"), usecols=[0, 1], skiprows=1)
df_covid["date"] = pd.to_datetime(df_covid["date"])
df_covid = df_covid.set_index("date")
df_covid

In [ ]:
df_covid.plot()

# 人々の流れ

Googleのモビリティレポートから取得したCSVファイルをもとに、

データの結合や不要な列の削除を行って使いやすくしたCSVファイルが以下にアップロードされている。

https://www.gifu-nct.ac.jp/elec/ohashi/datascience/mobilityreport.csv

元データは以下からダウンロードできる。

https://www.google.com/covid19/mobility/?hl=ja

それぞれのデータの説明は以下に掲載されている。

https://support.google.com/covid19-mobility/answer/9825414?hl=ja&ref_topic=9822927



それぞれのデータについて説明する。

`retail_and_recreation_percent_change_from_baseline`:

娯楽関連施設の、曜日別基準値（ 2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）

と比較した訪問者数の増加割合[%]

`transit_stations_percent_change_from_baseline`:

公共交通機関の、曜日別基準値と比較した訪問者数の増加割合[%]

`residential_percent_change_from_baseline`:

住居の、曜日別基準値と比較した滞在時間の増加割合[%]

住居のみ性質の異なるデータであることに注意する。

UNIXコマンド`curl`でGoogleドライブにファイルをダウンロードする。

In [ ]:
!curl -o /content/drive/MyDrive/mobility.csv https://www.gifu-nct.ac.jp/elec/ohashi/datascience/mobilityreport.csv

In [ ]:
df_mobility = pd.read_csv(os.path.join(drive_path, "mobility.csv"),
                          usecols=[0, 1, 4, 6])
df_mobility = df_mobility.set_axis(["date", "retail", "transit", "residential"],
                                   axis=1)
df_mobility["date"] = pd.to_datetime(df_mobility["date"])
df_mobility = df_mobility.set_index("date")
df_mobility

In [ ]:
df_mobility.plot()

# 相関関係の分析

上のデータから任意の2データを選び、相関関係を分析する。

まずはDataFrameを結合する。

`index`つまり`date`が共通の列だけを結合した新たなDataFrameを作る。

In [ ]:
# 2つのdfでindexが共通の列だけを結合する
df_concat = pd.concat([df_covid, df_mobility], axis=1, join="inner")
df_concat

今回は、第1波だけに着目する。

配列の**スライス**を使って2020-06-31までのデータを取り出す。

In [ ]:
df_concat = df_concat[:137]
df_concat

それぞれの変数をプロットしてみる。

In [ ]:
df_concat.plot(subplots=True)

グラフを見ると、感染者数の増加に遅れるような形で公共交通機関や娯楽店の訪問者が減少している。

この傾向を第2波や第3波などと比較してもよい。

また、時間遅れを加味した関連性を考える際には**相互相関**をとるとよい。

発表では相関分析や回帰分析以外の考察を説明してもよい。

matplotlibのラッパーである`seaborn`ライブラリを使って、

すべての組み合わせの2変数で散布図を作成できる。

In [ ]:
sns.pairplot(df_concat)

散布図を描くことで、2変数がどのように関係しているか直感的に確認できる。

この図をもとに考察してもよい。

これらの相関係数を求める。

3変数以上ある場合にも、相互に相関係数を求められる。

In [ ]:
corr = df_concat.corr()
sns.heatmap(corr, vmax=1, vmin=-1, center=0, 
            cmap="seismic", square=True, annot=True)

今回は`cases`と`retail`の関係に着目する。

感染者数が増えたとき、娯楽店の訪問者が減る負の相関があるといえる。

# 単回帰分析

任意の1変数を目的変数(y軸)、別の任意の1変数を説明変数(x軸)として、

単回帰分析を行う。

まず、機械学習のためのライブラリscikit-learnから

単回帰分析のためのモジュールを読み込む。

In [ ]:
from sklearn import linear_model

In [ ]:
clf = linear_model.LinearRegression()
X = df_concat[["cases"]]
Y = df_concat[["retail"]]
clf.fit(X, Y)

print("回帰係数: " + str(clf.coef_))
print("切片: " + str(clf.intercept_))
print("決定係数: " + str(clf.score(X, Y)))

決定係数は低いため、このモデルでは目的変数を説明できない。

回帰直線を散布図に重ねて描くと以下のようになる。

In [ ]:
sns.lmplot(x="cases", y="retail", data=df_concat[-365:])

グラフからも、単回帰によって十分に目的変数を説明できないことがわかる。